In [96]:
with open("day19.txt", "r") as f:
    lines = [line.strip() for line in f.readlines()]

In [97]:
lines

['rr{s>898:A,s>537:A,a>2567:R,A}',
 'gmr{x<1782:A,x<1819:R,R}',
 'dln{m<1743:R,R}',
 'kd{x>1248:xsx,m<797:cxm,ppz}',
 'lsv{m<2789:R,A}',
 'ld{a>2638:A,x<2418:tsb,sds}',
 'bb{x<2091:gb,s>2804:lxr,vz}',
 'rjc{s<2734:R,x<709:R,R}',
 'ng{x<3090:A,a<3366:R,a>3456:R,R}',
 'fdh{a<2472:jbt,chz}',
 'pg{x<3179:A,x>3508:R,s<2653:R,A}',
 'ckl{x<1338:R,s>1155:R,R}',
 'tt{m<3331:R,x<2654:R,a<2193:R,A}',
 'vk{x>1759:hzr,xgr}',
 'pgn{s>3747:A,R}',
 'ksl{a>1008:dq,s>1178:vc,a<669:A,A}',
 'frt{s<2175:R,x<3670:R,R}',
 'hk{x>767:jn,lvr}',
 'dnq{s<1989:A,A}',
 'rcp{a>1871:A,x<879:A,s>3196:A,A}',
 'vdc{s>1410:A,x<2648:R,R}',
 'hp{x>3237:R,R}',
 'sps{s<2626:A,A}',
 'fsz{a>1966:R,R}',
 'dtd{x<3224:A,x<3493:njp,R}',
 'nzm{s>1996:R,s>1738:R,R}',
 'shm{m<3138:A,m>3515:A,A}',
 'rbl{s>2903:A,A}',
 'rqr{a<1658:cz,a>2538:hr,cr}',
 'mfd{m<2895:bkh,m>3041:R,x>605:R,kpt}',
 'cxm{x<733:fqb,m<305:lh,A}',
 'mtj{a>682:A,m<514:R,R}',
 'nqp{x<1742:xmh,m<2426:qg,a<1664:zn,mdq}',
 'qbc{s>1746:R,a>3471:A,A}',
 'gcb{a>3873:R,x>9

In [98]:
workflows = {}
artifacts = []

for line in lines:
    if line == "":
        continue
    if not line.startswith("{"):
        # workflow
        workflow_name, workflow_step_str = line.replace("}","").split("{")
        workflow_steps = workflow_step_str.split(",")
        steps = []
        for step in workflow_steps:
            if ":" not in step:
                dim, comp, val, action = "always", "always", "always", step
            else:
                condition, action = step.split(":")
                dim, comp, val = condition[0], condition[1], int(condition[2:])
            steps.append((dim, comp, val, action))    
        workflows[workflow_name] = steps
    else:
        # artifact
        line = line.replace("{","").replace("}","")
        values = line.split(",")
        artifact = {}
        for value in values:
            dim, val = value.split("=")
            val = int(val)
            artifact[dim] = val
        artifacts.append(artifact)

In [99]:
workflows

{'rr': [('s', '>', 898, 'A'),
  ('s', '>', 537, 'A'),
  ('a', '>', 2567, 'R'),
  ('always', 'always', 'always', 'A')],
 'gmr': [('x', '<', 1782, 'A'),
  ('x', '<', 1819, 'R'),
  ('always', 'always', 'always', 'R')],
 'dln': [('m', '<', 1743, 'R'), ('always', 'always', 'always', 'R')],
 'kd': [('x', '>', 1248, 'xsx'),
  ('m', '<', 797, 'cxm'),
  ('always', 'always', 'always', 'ppz')],
 'lsv': [('m', '<', 2789, 'R'), ('always', 'always', 'always', 'A')],
 'ld': [('a', '>', 2638, 'A'),
  ('x', '<', 2418, 'tsb'),
  ('always', 'always', 'always', 'sds')],
 'bb': [('x', '<', 2091, 'gb'),
  ('s', '>', 2804, 'lxr'),
  ('always', 'always', 'always', 'vz')],
 'rjc': [('s', '<', 2734, 'R'),
  ('x', '<', 709, 'R'),
  ('always', 'always', 'always', 'R')],
 'ng': [('x', '<', 3090, 'A'),
  ('a', '<', 3366, 'R'),
  ('a', '>', 3456, 'R'),
  ('always', 'always', 'always', 'R')],
 'fdh': [('a', '<', 2472, 'jbt'), ('always', 'always', 'always', 'chz')],
 'pg': [('x', '<', 3179, 'A'),
  ('x', '>', 3508, 'R

In [100]:
# build a graph of workflows?
# how to reverse...
# bruteforce is definitely not an option...

In [101]:
def get_reverse_of_step(step):
    dim, comp, val, action = step
    if comp == ">":
        return (dim, '<=', val, action)
    elif comp == "<":
        return (dim, '>=', val, action)
    elif comp == "always":
        return ("never", "never", "never", action)

In [102]:
# convert_into_edges
edge_map = {}
for workflow_name, steps in workflows.items():
    edge_map[workflow_name] = []
    for idx, step in enumerate(steps):
        current_conditions = []
        for prev_step in steps[:idx]:
            rev_dim, rev_comp, rev_val, rev_action = get_reverse_of_step(prev_step)
            current_conditions.append((rev_dim, rev_comp, rev_val))
        dim, comp, val, action = step
        if dim != "always":
            current_conditions.append((dim,comp,val))
    
        edge_map[workflow_name].append((action, current_conditions))

In [103]:
edge_map

{'rr': [('A', [('s', '>', 898)]),
  ('A', [('s', '<=', 898), ('s', '>', 537)]),
  ('R', [('s', '<=', 898), ('s', '<=', 537), ('a', '>', 2567)]),
  ('A', [('s', '<=', 898), ('s', '<=', 537), ('a', '<=', 2567)])],
 'gmr': [('A', [('x', '<', 1782)]),
  ('R', [('x', '>=', 1782), ('x', '<', 1819)]),
  ('R', [('x', '>=', 1782), ('x', '>=', 1819)])],
 'dln': [('R', [('m', '<', 1743)]), ('R', [('m', '>=', 1743)])],
 'kd': [('xsx', [('x', '>', 1248)]),
  ('cxm', [('x', '<=', 1248), ('m', '<', 797)]),
  ('ppz', [('x', '<=', 1248), ('m', '>=', 797)])],
 'lsv': [('R', [('m', '<', 2789)]), ('A', [('m', '>=', 2789)])],
 'ld': [('A', [('a', '>', 2638)]),
  ('tsb', [('a', '<=', 2638), ('x', '<', 2418)]),
  ('sds', [('a', '<=', 2638), ('x', '>=', 2418)])],
 'bb': [('gb', [('x', '<', 2091)]),
  ('lxr', [('x', '>=', 2091), ('s', '>', 2804)]),
  ('vz', [('x', '>=', 2091), ('s', '<=', 2804)])],
 'rjc': [('R', [('s', '<', 2734)]),
  ('R', [('s', '>=', 2734), ('x', '<', 709)]),
  ('R', [('s', '>=', 2734), ('

In [104]:
# try to do bfs until A?

all_accepted_paths = []

queue = []
queue.append(("in", [], []))

iter_count = 0

while len(queue) > 0 and iter_count < 100000:
    iter_count += 1
    current_node, current_path, current_conditions = queue.pop(0)
    if current_node == "A":
        all_accepted_paths.append((current_path, current_conditions))
        continue
    if current_node == "R":
        # rejected
        continue
    for next_node, conditions in edge_map[current_node]:
        if next_node in current_path:
            # prevent loops
            print("inside loops?, current_path", current_path, "next_node", next_node)
            continue
        queue.append((next_node, current_path + [next_node], current_conditions + conditions))

In [105]:
len(all_accepted_paths)

575

In [106]:
all_accepted_paths[0]

(['qfr', 'dk', 'ngm', 'ck', 'cx', 'A'],
 [('s', '<', 1496),
  ('m', '<', 1406),
  ('m', '<', 793),
  ('m', '>', 464),
  ('m', '>', 591),
  ('x', '<=', 1442)])

In [107]:
all_accepted_paths[-1]

(['nqp', 'mdq', 'fk', 'vpk', 'fqr', 'zhv', 'A'],
 [('s', '>=', 1496),
  ('m', '>=', 1482),
  ('x', '>=', 1742),
  ('m', '>=', 2426),
  ('a', '>=', 1664),
  ('s', '<=', 2571),
  ('m', '<=', 3381),
  ('x', '>=', 2597),
  ('x', '<=', 3474),
  ('m', '>=', 2927),
  ('s', '>=', 2002),
  ('a', '<=', 2845),
  ('a', '<=', 2145),
  ('a', '<=', 1975),
  ('m', '<=', 3165),
  ('x', '<', 3102)])

In [108]:
from functools import reduce

In [109]:
def merge_conditions(conditions):

    # first merge all conditions with the same dimension
    merged = {}
    for dim, comp, val in conditions:
        if dim not in merged:
            merged[dim] = []
        merged[dim].append((comp, val))

    # then calculate the true accepted interval
    ok_counts = {
        "x": 4000,
        "m": 4000,
        "a": 4000,
        "s": 4000,
    }
    for dim, comp_vals in merged.items():
        ok = 0
        for i in range(1,4001):
            comp_ok = True
            for comp, val in comp_vals:
                if comp == ">=" and i >= val:
                    pass
                elif comp == "<=" and i <= val:
                    pass
                elif comp == ">" and i > val:
                    pass
                elif comp == "<" and i < val:
                    pass
                else:
                    comp_ok = False
                    break
            if comp_ok:
                ok += 1
        ok_counts[dim] = ok
    print(ok_counts)
    return reduce(lambda x,y: x*y, ok_counts.values())

In [110]:
total_count = 0
for path, conditions in all_accepted_paths:
    count = merge_conditions(conditions)
    total_count += count
    print(path, conditions, count)
total_count


{'x': 1442, 'm': 201, 'a': 4000, 's': 1495}
['qfr', 'dk', 'ngm', 'ck', 'cx', 'A'] [('s', '<', 1496), ('m', '<', 1406), ('m', '<', 793), ('m', '>', 464), ('m', '>', 591), ('x', '<=', 1442)] 1733255160000
{'x': 1965, 'm': 127, 'a': 4000, 's': 1495}
['qfr', 'dk', 'ngm', 'ck', 'mb', 'A'] [('s', '<', 1496), ('m', '<', 1406), ('m', '<', 793), ('m', '>', 464), ('m', '<=', 591), ('x', '<', 1966)] 1492338900000
{'x': 2302, 'm': 109, 'a': 2047, 's': 1495}
['qfr', 'dk', 'ngm', 'gn', 'dhv', 'A'] [('s', '<', 1496), ('m', '<', 1406), ('m', '<', 793), ('m', '<=', 464), ('a', '<', 2048), ('x', '>', 1698), ('m', '<=', 237), ('m', '>=', 129)] 767875573270
{'x': 1698, 'm': 464, 'a': 2047, 's': 695}
['qfr', 'dk', 'ngm', 'gn', 'kgb', 'A'] [('s', '<', 1496), ('m', '<', 1406), ('m', '<', 793), ('m', '<=', 464), ('a', '<', 2048), ('x', '<=', 1698), ('s', '<', 696)] 1120877918880
{'x': 1121, 'm': 464, 'a': 1252, 's': 1495}
['qfr', 'dk', 'ngm', 'qd', 'ts', 'A'] [('s', '<', 1496), ('m', '<', 1406), ('m', '<', 79

130090458884662

: 